# 4. Capstone 3 Pre-Processing & Modeling

In [6]:
!pip install tensorflow==1.2.0 --ignore-installed

  Using cached numpy-1.19.5-cp36-cp36m-win_amd64.whl (13.2 MB)
  Using cached protobuf-3.17.3-cp36-cp36m-win_amd64.whl (910 kB)
  Using cached wheel-0.36.2-py2.py3-none-any.whl (35 kB)
  Using cached dataclasses-0.8-py3-none-any.whl (19 kB)
  Created wheel for html5lib: filename=html5lib-0.9999999-py3-none-any.whl size=107234 sha256=64f5c1bd6e057a8f30150cc7ece9eec85eb741c84f051623aa40dcfeef3e66c8
  Stored in directory: c:\users\allegra grunberg\appdata\local\pip\cache\wheels\90\1c\cb\a87fd097ff74648ecc468a703001f6c7c86d8a71d459e65c98
  Created wheel for markdown: filename=Markdown-2.2.0-py3-none-any.whl size=136943 sha256=2a75c4f1f09ac7c4a35fbc2a0e40f40c66327569564cee59e8af473bdeed112a
  Stored in directory: c:\users\allegra grunberg\appdata\local\pip\cache\wheels\fa\d9\f4\64de9f84b06e91e24034ba45fb0a3472ae718a5cd81a172c99
Successfully built html5lib markdown


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\dev\\Lib\\site-packages\\numpy\\.libs\\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the 'c:\dev\python.exe -m pip install --upgrade pip' command.


In [3]:
import pandas as pd
import numpy as np

In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint

import transformers
from tqdm.notebook import tqdm
from tokenizers import BertWordPieceTokenizer

# Install spaCy 
!pip3 install -U spacy

# Download the large English model for spaCy
!pip3 install -m spacy download en_core_web_lg

# Install textacy which will also be useful
!pip3 install -U textacy 

ModuleNotFoundError: No module named 'tensorflow'

In [5]:
wall = pd.read_csv('reddit_wsb.csv')

In [9]:
wall.head()

,title,score,id,url,comms_num,created,body,timestamp
0,"It's not about the money, it's about sending a...",55,l6ulcx,https://v.redd.it/6j75regs72e61,6,1.611863e+09,NaN,2021-01-28 21:37:41
1,Math Professor Scott Steiner says the numbers ...,110,l6uibd,https://v.redd.it/ah50lyny62e61,23,1.611862e+09,NaN,2021-01-28 21:32:10
2,Exit the system,0,l6uhhn,https://www.reddit.com/r/wallstreetbets/commen...,47,1.611862e+09,The CEO of NASDAQ pushed to halt trading “to g...,2021-01-28 21:30:35
3,NEW SEC FILING FOR GME! CAN SOMEONE LESS RETAR...,29,l6ugk6,https://sec.report/Document/0001193125-21-019848/,74,1.611862e+09,NaN,2021-01-28 21:28:57
4,"Not to distract from GME, just thought our AMC...",71,l6ufgy,https://i.redd.it/4h2sukb662e61.jpg,156,1.611862e+09,NaN,2021-01-28 21:26:56


# Pre-Processing

In [17]:
#test training splitting 
from sklearn.model_selection import train_test_split

X = wall.drop('body', axis=1)
y = wall['body']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=27)

In [15]:
X.shape, y.shape

((37155, 7), (37155,))

In [22]:
# Use the trained to create a document-term matrix from train and test sets
x_train_dtm = vect.transform(X_train)
x_test_dtm = vect.transform(X_test)

(TF-IDF)

Clean texts into numeric features.

In [24]:
#might need this for modeling--machine learning deep learning etc 

from sklearn.feature_extraction.text import TfidfTransformer

#add variable 

tfidf_transformer = TfidfTransformer()

tfidf_transformer.fit(x_train_dtm)
x_train_tfidf = tfidf_transformer.transform(x_train_dtm)

x_train_tfidf

<7x7 sparse matrix of type '<class 'numpy.float64'>'
	with 7 stored elements in Compressed Sparse Row format>

Vectorization

In [19]:
from sklearn.feature_extraction.text import CountVectorizer

# instantiate the vectorizer
vect = CountVectorizer()
vect.fit(X_train)

CountVectorizer()

In [21]:
wall.head()

,title,score,id,url,comms_num,created,body,timestamp
0,"It's not about the money, it's about sending a...",55,l6ulcx,https://v.redd.it/6j75regs72e61,6,1.611863e+09,NaN,2021-01-28 21:37:41
1,Math Professor Scott Steiner says the numbers ...,110,l6uibd,https://v.redd.it/ah50lyny62e61,23,1.611862e+09,NaN,2021-01-28 21:32:10
2,Exit the system,0,l6uhhn,https://www.reddit.com/r/wallstreetbets/commen...,47,1.611862e+09,The CEO of NASDAQ pushed to halt trading “to g...,2021-01-28 21:30:35
3,NEW SEC FILING FOR GME! CAN SOMEONE LESS RETAR...,29,l6ugk6,https://sec.report/Document/0001193125-21-019848/,74,1.611862e+09,NaN,2021-01-28 21:28:57
4,"Not to distract from GME, just thought our AMC...",71,l6ufgy,https://i.redd.it/4h2sukb662e61.jpg,156,1.611862e+09,NaN,2021-01-28 21:26:56


# Modeling 

- Precision: The positive predictive power of a model. Out of all the predictions made by a model for a class, how many are actually correct
- Recall: The coverage or hit-rate of a model. Out of all the test data samples belonging to a class, how many was the model able to predict (hit or cover) correctly.
- F1-score: The harmonic mean of the precision and recall
- Do check out ROC Curve, AUC Score and PR Curve also

https://www.analyticsvidhya.com/blog/2020/10/reinforcement-learning-stock-price-prediction/#:~:text=ARIMA%20is%20one%20such%20model,networks%20for%20predicting%20continuous%20values.

#FYI do not do one hot encoding this is not categorical data and thus is useless and cumbersome - memory intensive 

In [ ]:
lr.fit(X_train.drop(['Review'], axis=1), y_train)
predictions = lr.predict(X_test.drop(['Review'], axis=1))
meu.display_classification_report(true_labels=y_test, predicted_labels=predictions)

Logistic Regression 

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(C=1, random_state=42)

In [ ]:
# Logistic Regression model on BOW features
from sklearn.linear_model import LogisticRegression

# instantiate model
lr = LogisticRegression(penalty='l2', max_iter=500, C=1, solver='lbfgs', random_state=42)

# train model
lr.fit(cv_train_features, train_sentiments)

# predict on test data
lr_bow_predictions = lr.predict(cv_test_features)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

labels = ['negative', 'positive']
print(classification_report(test_sentiments, lr_bow_predictions))
pd.DataFrame(confusion_matrix(test_sentiments, lr_bow_predictions), index=labels, columns=labels)

In [ ]:
Random Forest

# Random Forest model on BOW features
from sklearn.ensemble import RandomForestClassifier

# instantiate model
rf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=42)

# train model
rf.fit(cv_train_features, train_sentiments)

# predict on test data
rf_bow_predictions = rf.predict(cv_test_features)

In [ ]:
labels = ['negative', 'positive']
print(classification_report(test_sentiments, rf_bow_predictions))
pd.DataFrame(confusion_matrix(test_sentiments, rf_bow_predictions), index=labels, columns=labels)

Gradiant Boosting

Summary 

# Sentiment Analysis- MOVE 
- http://localhost:8889/notebooks/Downloads/Springboard/Capstone%203/%20Old%20Capstone%203/Craft%20a%20Story-Project-20.3/Capstone%203-%20Wallstreet%20NLP-26.2-%20Springboard.ipynb

#https://nbviewer.jupyter.org/github/dipanjanS/practical_nlp_workshop_gids20/blob/main/tutorials/03_NLP%20Applications/09_NLP_Applications_Text_Classification_Machine_Learning_and_Basic_DNN.ipynb

In [ ]:
headlines = []
for row in range(0,len(data.index)):
    headlines.append(' '.join(str(x) for x in data.iloc[row,0:25]))

Vectorizer 

In [ ]:
## implement BAG OF WORDS
countvector=CountVectorizer(ngram_range=(2,2))
traindataset=countvector.fit_transform(headlines)

In [ ]:
# 0-1 sparse matrix
traindataset[0]

Random Forrest

In [ ]:
# implement RandomForest Classifier
randomclassifier=RandomForestClassifier(n_estimators=200,criterion='entropy')
randomclassifier.fit(traindataset,train['Label'])

Test data preprocessing and Prediction

In [ ]:
Test data preprocessing and Prediction

## Predict for the Test Dataset
test_transform= []
for row in range(0,len(test.index)):
    test_transform.append(' '.join(str(x) for x in test.iloc[row,2:27]))
test_dataset = countvector.transform(test_transform)
predictions = randomclassifier.predict(test_dataset)

Evaluation of the model

In [ ]:
## Import library to check accuracy
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
matrix=confusion_matrix(test['Label'],predictions)
print(matrix)
score=accuracy_score(test['Label'],predictions)
print(score)
report=classification_report(test['Label'],predictions)
print(report)

https://medium.com/analytics-vidhya/using-sentiment-analysis-to-predict-the-stock-market-77100295d753

Reviews are pretty subjective, opinionated and people often express stong emotions, feelings. This makes it a classic case where the text documents here are a good candidate for extracting sentiment as a feature.

TextBlob is an excellent open-source library for performing NLP tasks with ease, including sentiment analysis. It also an a sentiment lexicon (in the form of an XML file) which it leverages to give both polarity and subjectivity scores.

The polarity score is a float within the range [-1.0, 1.0].
The subjectivity is a float within the range [0.0, 1.0] where 0.0 is very objective and 1.0 is very subjective.

In [ ]:
textblob.TextBlob('This is an AMAZING pair of Jeans!').sentiment

Remember this is unsupervised, lexicon-based sentiment analysis where we don't have any pre-labeled data saying which review migth have a positive or negative sentiment. We use the lexicon to determine this.

In [ ]:
x_train_snt_obj = X_train['Review'].apply(lambda row: textblob.TextBlob(row).sentiment)
X_train['Polarity'] = [obj.polarity for obj in x_train_snt_obj.values]
X_train['Subjectivity'] = [obj.polarity for obj in x_train_snt_obj.values]

x_test_snt_obj = X_test['Review'].apply(lambda row: textblob.TextBlob(row).sentiment)
X_test['Polarity'] = [obj.polarity for obj in x_test_snt_obj.values]
X_test['Subjectivity'] = [obj.polarity for obj in x_test_snt_obj.values]

In [ ]:
X_train.head()

In [ ]:
lr.fit(X_train.drop(['Review'], axis=1), y_train)
predictions = lr.predict(X_test.drop(['Review'], axis=1))
meu.display_classification_report(true_labels=y_test, predicted_labels=predictions)

Experiment 4: Adding Bag of Words based Features - 1-grams
This is perhaps the most simple vector space representational model for unstructured text. A vector space model is simply a mathematical model to represent unstructured text (or any other data) as numeric vectors, such that each dimension of the vector is a specific feature\attribute.

The bag of words model represents each text document as a numeric vector where each dimension is a specific word from the corpus and the value could be its frequency in the document, occurrence (denoted by 1 or 0) or even weighted values.

The model’s name is such because each document is represented literally as a ‘bag’ of its own words, disregarding word orders, sequences and grammar.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(min_df=0.0, max_df=1.0, ngram_range=(1, 1))
X_traincv = cv.fit_transform(X_train['Clean Review']).toarray()
X_traincv = pd.DataFrame(X_traincv, columns=cv.get_feature_names())

X_testcv = cv.transform(X_test['Clean Review']).toarray()
X_testcv = pd.DataFrame(X_testcv, columns=cv.get_feature_names())
X_traincv.head()

In [ ]:
X_train_comb = pd.concat([X_train_metadata, X_traincv], axis=1)
X_test_comb = pd.concat([X_test_metadata, X_testcv], axis=1)

X_train_comb.head()

In [ ]:
lr.fit(X_train_comb, y_train)
predictions = lr.predict(X_test_comb)
meu.display_classification_report(true_labels=y_test, predicted_labels=predictions)

In [ ]:
Experiment 5: Adding Bag of Words based Features - 2-grams
We use the same feature engineering technique here except we consider both 1 and 2-grams as our features. We also do some basic filtering like removing terms which might occur only once or almost in every document!

In [ ]:
cv = CountVectorizer(min_df=2, max_df=0.99, ngram_range=(1, 2))
X_traincv = cv.fit_transform(X_train['Clean Review']).toarray()
X_traincv = pd.DataFrame(X_traincv, columns=cv.get_feature_names())

X_testcv = cv.transform(X_test['Clean Review']).toarray()
X_testcv = pd.DataFrame(X_testcv, columns=cv.get_feature_names())
X_traincv.head()

In [ ]:
X_train_comb = pd.concat([X_train_metadata, X_traincv], axis=1)
X_test_comb = pd.concat([X_test_metadata, X_testcv], axis=1)

X_train_comb.head()

In [ ]:
lr.fit(X_train_comb, y_train)
predictions = lr.predict(X_test_comb)
meu.display_classification_report(true_labels=y_test, predicted_labels=predictions)

In [ ]:
add above info to model / sentiment is seperate

In [ ]:
#sentiment analysis body of texts

# borrowed from https://www.kaggle.com/pashupatigupta/sentiments-transformer-vader-embedding-bert
sia = SentimentIntensityAnalyzer()
def find_sentiment(post):
    if sia.polarity_scores(post)["compound"] > 0:
        return "Positive"
    elif sia.polarity_scores(post)["compound"] < 0:
        return "Negative"
    else:
        return "Neutral"       
    
def plot_sentiment(df, feature, title):
    counts = df[feature].value_counts()
    percent = counts/sum(counts)

    fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12, 5))

    counts.plot(kind='bar', ax=ax1, color='green')
    percent.plot(kind='bar', ax=ax2, color='blue')
    ax1.set_ylabel(f'Counts : {title} sentiments', size=12)
    ax2.set_ylabel(f'Percentage : {title} sentiments', size=12)
    plt.suptitle(f"Sentiment analysis: {title}")
    plt.tight_layout()
    plt.show()    def plot_sentiment(df, feature, title):
    counts = df[feature].value_counts()
    percent = counts/sum(counts)

    fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12, 5))

    counts.plot(kind='bar', ax=ax1, color='green')
    percent.plot(kind='bar', ax=ax2, color='blue')
    ax1.set_ylabel(f'Counts : {title} sentiments', size=12)
    ax2.set_ylabel(f'Percentage : {title} sentiments', size=12)
    plt.suptitle(f"Sentiment analysis: {title}")
    plt.tight_layout()
    plt.show()
    
df = data_df.loc[~data_df.body.isna()]
df['body_sentiment'] = df['body'].apply(lambda x: find_sentiment(x))
plot_sentiment(df, 'body_sentiment', 'Body')  

In [ ]:
#sentiment correlation of body text 

plt.figure(figsize=(12,10))
sns.heatmap(df.groupby('title_sentiment').corr(), cmap = 'viridis')
plt.title("Title Correlation")

Run sentiment analysis and calculate a score

In [ ]:
init_.pd_data.plot(x =init_.sentiment_name,y=init_.daily_return,style = "o")

In [ ]:
#We also calculate the correlation between these two

print(init_.pd_data[init_.daily_return].corr(init_.pd_data[init_.sentiment_name]))

In [ ]:
if __name__ == '__main__':
    init_ = Init()
    for ticker in init_.tickers:
        fig, ax = plt.subplots() # Create the figure and axes object
        hist_ret = HistoricalReturn(start_date=init_.start_date, end_date=init_.end_date, start_date_=init_.start_date_,
                                  end_date_=init_.end_date_, ticker=ticker, dir_path=init_.dir_path,
                                  db_name=init_.db_name,daily_return=init_.daily_return)
        init_.pd_data = hist_ret.historical_price()

        v_analysis = VaderAnalysis(hist_price = init_.pd_data,start_date=init_.start_date, end_date=init_.end_date,
                                   start_date_=init_.start_date_,end_date_=init_.end_date_, ticker=ticker + '_',
                                   dir_path=init_.dir_path,db_name=init_.db_name, news_header = init_.news_header,
                                   sentiment_name=init_.sentiment_name)
        init_.pd_data = v_analysis.vader_analysis()
        init_.pd_data.plot(x =init_.sentiment_name,y=init_.daily_return,style = "o")
        print(init_.pd_data[init_.daily_return].corr(init_.pd_data[init_.sentiment_name]))

Topic Modeling 

One such technique in the field of text mining is Topic Modelling. As the name suggests, it is a process to automatically identify topics present in a text object and to derive hidden patterns exhibited by a text corpus. Thus, assisting better decision making.

1. https://www.analyticsvidhya.com/blog/2016/08/beginners-guide-to-topic-modeling-in-python/
2. https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24

In [ ]:
import nltk
import contractions
import re
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

In [ ]:
#Latent Dirichlet Allocation for Topic Modeling

from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

doc_clean = [clean(doc).split() for doc in doc_complete] 

In [ ]:
#Running LDA Model
https://towardsdatascience.com/nlp-extracting-the-main-topics-from-your-dataset-using-lda-in-minutes-21486f5aa925 

# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=3, id2word = dictionary, passes=50)

In [ ]:
#Preparing Document-Term Matrix

# Importing Gensim
import gensim
from gensim import corpora

# Creating the term dictionary of our courpus, where every unique term is assigned an index. dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [ ]:
print(ldamodel.print_topics(num_topics=3, num_words=3))
['0.168*health + 0.083*sugar + 0.072*bad,
'0.061*consume + 0.050*drive + 0.050*sister,
'0.049*pressur + 0.049*father + 0.049*sister]

# Deep Learning 

ddont do this unless you have time 

In [ ]:
import gensim
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Activation, Dense
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Create a Multinomial Naive Bayes model
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

# Train the model
nb.fit(x_train_dtm, y_train)

In [ ]:
#Naive Bayes DTM

# Make class anf probability predictions
y_pred_class = nb.predict(x_test_dtm)
y_pred_prob = nb.predict_proba(x_test_dtm)[:, 1]

# calculate accuracy of class predictions
from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred_class))

conf_matrix(metrics.confusion_matrix(y_test, y_pred_class))

# Calculate AUC
metrics.roc_auc_score(y_test, y_pred_prob)

In [ ]:
#Native Bayes

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

pipe = Pipeline([('bow', CountVectorizer()), 
                 ('tfid', TfidfTransformer()),  
                 ('model', MultinomialNB())])

# Fit the pipeline with the data
pipe.fit(x_train, y_train)

y_pred_class = pipe.predict(x_test)

print(metrics.accuracy_score(y_test, y_pred_class))

conf_matrix(metrics.confusion_matrix(y_test, y_pred_class))

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')

def bert_encode(data, maximum_length) :
    input_ids = []
    attention_masks = []

    for text in data:
        encoded = tokenizer.encode_plus(
            text, 
            add_special_tokens=True,
            max_length=maximum_length,
            pad_to_max_length=True,

            return_attention_mask=True,
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
        
    return np.array(input_ids),np.array(attention_masks)

Convolutional Neural Networks (CNNs)

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

# fix random seed for reproducibility
seed = 42
np.random.seed(seed)

In [ ]:
import contractions
from bs4 import BeautifulSoup
import numpy as np
import re
import tqdm
import unicodedata


def strip_html_tags(text):
  soup = BeautifulSoup(text, "html.parser")
  [s.extract() for s in soup(['iframe', 'script'])]
  stripped_text = soup.get_text()
  stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
  return stripped_text

def remove_accented_chars(text):
  text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
  return text

def pre_process_corpus(docs):
  norm_docs = []
  for doc in tqdm.tqdm(docs):
    doc = strip_html_tags(doc)
    doc = doc.translate(doc.maketrans("\n\t\r", "   "))
    doc = doc.lower()
    doc = remove_accented_chars(doc)
    doc = contractions.fix(doc)
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = re.sub(' +', ' ', doc)
    doc = doc.strip()  
    norm_docs.append(doc)
  
  return norm_docs

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

train_lens = [len(s) for s in train_sequences]
test_lens = [len(s) for s in test_sequences]

fig, ax = plt.subplots(1,2, figsize=(12, 6))
h1 = ax[0].hist(train_lens)
h2 = ax[1].hist(test_lens)

Sequence Normalization

In [ ]:
# Install spaCy 
!pip3 install -U spacy

# Download the large English model for spaCy
!pip3 install -m spacy download en_core_web_lg

# Install textacy which will also be useful
!pip3 install -U textacy

Bert

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')

def bert_encode(data, maximum_length) :
    input_ids = []
    attention_masks = []

    for text in data:
        encoded = tokenizer.encode_plus(
            text, 
            add_special_tokens=True,
            max_length=maximum_length,
            pad_to_max_length=True,

            return_attention_mask=True,
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
        
    return np.array(input_ids),np.array(attention_masks)